In [1]:
import logging
logging.root.handlers = []  # Jupyter messes up logging so needs a reset
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
from smart_open import smart_open
import pandas as pd
import numpy as np
from numpy import random
import gensim
import nltk
from sklearn.cross_validation import train_test_split
from sklearn import linear_model
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
from gensim.models import Word2Vec
from sklearn.neighbors import KNeighborsClassifier
from sklearn import linear_model
from nltk.corpus import stopwords
import os, codecs
%matplotlib inline
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords
from sklearn import metrics as skmetrics

2017-06-09 15:34:20,688 : INFO : 'pattern' package found; tag filters are available for English
/usr/lib64/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


## Model 1 evaluation

In [2]:
model = gensim.models.Word2Vec.load('/home/bahbbc/workspace/masters-big5/models/tweet50-600.model')
model.init_sims(replace=True)

2017-06-09 15:34:24,779 : INFO : loading Word2Vec object from /home/bahbbc/workspace/masters-big5/models/tweet50-600.model
2017-06-09 15:34:31,030 : INFO : loading wv recursively from /home/bahbbc/workspace/masters-big5/models/tweet50-600.model.wv.* with mmap=None
2017-06-09 15:34:31,031 : INFO : loading syn0 from /home/bahbbc/workspace/masters-big5/models/tweet50-600.model.wv.syn0.npy with mmap=None
2017-06-09 15:34:39,044 : INFO : setting ignored attribute syn0norm to None
2017-06-09 15:34:39,045 : INFO : loading syn1neg from /home/bahbbc/workspace/masters-big5/models/tweet50-600.model.syn1neg.npy with mmap=None
2017-06-09 15:35:10,125 : INFO : setting ignored attribute cum_table to None
2017-06-09 15:35:11,101 : INFO : loaded /home/bahbbc/workspace/masters-big5/models/tweet50-600.model
2017-06-09 15:36:40,519 : INFO : precomputing L2-norms of word weight vectors


In [3]:
num_features= 600

In [4]:
df = pd.read_csv('~/personality-normalized-word2vec-norm.csv', encoding='utf-8')
df.shape

(1039, 186)

In [5]:
df.neuroticism_m.value_counts()

0    535
1    504
Name: neuroticism_m, dtype: int64

In [6]:
train_w2v_data, test_w2v_data = train_test_split(df, test_size=0.3, random_state=42)

In [7]:
def predict(vectorizer, classifier, data):
    data_features = vectorizer.transform(data['formatted_text'])
    predictions = classifier.predict(data_features)
    target = data['neuroticism_m']
    evaluate_prediction(predictions, target)

In [8]:
def word_averaging(wv, words):
    all_words, mean = set(), []
    #print words.shape
    for word in words:
        if isinstance(word, np.ndarray):
            mean.append(word)
        elif word in wv.wv.vocab:
            mean.append(wv.wv.syn0norm[wv.wv.vocab[word].index])
            all_words.add(wv.wv.vocab[word].index)
        #print mean

    if not mean:
        logging.warning("cannot compute similarity with no input %s", words)
        # FIXME: remove these examples in pre-processing
        return np.zeros(num_features,)

    mean = gensim.matutils.unitvec(np.array(mean).mean(axis=0)).astype(np.float32)
    return mean

def  word_averaging_list(wv, text_list):
    return np.vstack([word_averaging(wv, review) for review in text_list ])

In [9]:
def w2v_tokenize_text(text):
    tokens = []
    if text is np.nan:
        return []
    for sent in nltk.sent_tokenize(text, language='portuguese'):
        for word in nltk.word_tokenize(sent, language='portuguese'):
            if len(word) < 2:
                continue
            tokens.append(word)
    return tokens

In [10]:
test_tokenized = test_w2v_data.apply(lambda r: w2v_tokenize_text(r['formatted_text']), axis=1).values
train_tokenized = train_w2v_data.apply(lambda r: w2v_tokenize_text(r['formatted_text']), axis=1).values

In [11]:
%%time
X_train_word_average = word_averaging_list(model,train_tokenized)
X_test_word_average = word_averaging_list(model,test_tokenized)

2017-06-09 15:39:17,281 : WARNING : cannot compute similarity with no input []
2017-06-09 15:39:17,883 : WARNING : cannot compute similarity with no input []
2017-06-09 15:39:17,892 : WARNING : cannot compute similarity with no input []
2017-06-09 15:39:18,884 : WARNING : cannot compute similarity with no input []
2017-06-09 15:39:19,233 : WARNING : cannot compute similarity with no input []
2017-06-09 15:39:19,823 : WARNING : cannot compute similarity with no input []
2017-06-09 15:39:20,836 : WARNING : cannot compute similarity with no input []
2017-06-09 15:39:20,960 : WARNING : cannot compute similarity with no input []
2017-06-09 15:39:21,261 : WARNING : cannot compute similarity with no input []
2017-06-09 15:39:21,473 : WARNING : cannot compute similarity with no input []
2017-06-09 15:39:21,545 : WARNING : cannot compute similarity with no input []


CPU times: user 5.28 s, sys: 289 ms, total: 5.57 s
Wall time: 8.35 s


In [12]:
del model

In [13]:
X_train_word_average.shape

(727, 600)

In [14]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.externals import joblib

force = False
model_trainer = RandomizedSearchCV(
    n_iter=1, 
    estimator=RandomForestClassifier(),
    param_distributions={
        "criterion": ["gini"],
        "n_estimators": [1000],
        "max_features": ["log2"],
        "max_depth": [None],
        "bootstrap": [True],
        "oob_score": [True],
        "class_weight": ["balanced"],
        "random_state": [42]
    },
    scoring="f1",
    verbose=True,
    refit=True,
    cv=10,
    n_jobs=-1
)

In [15]:
%%time
model_trainer.fit(X_train_word_average, train_w2v_data['neuroticism_m'])
model = model_trainer.best_estimator_

Fitting 10 folds for each of 1 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   26.1s finished


CPU times: user 5.38 s, sys: 409 ms, total: 5.79 s
Wall time: 33.6 s


In [16]:
yp = model.predict(X_test_word_average)
yt = test_w2v_data['neuroticism_m']

In [17]:
pd.DataFrame(
    index=pd.Index([0, 1], name="y_true"),
    columns=pd.Index([0, 1], name="y_pred"),
    data=skmetrics.confusion_matrix(y_true=yt, y_pred=yp)
)

y_pred,0,1
y_true,,
0,96,67
1,81,68


In [18]:
print skmetrics.classification_report(y_true=yt, y_pred=yp)

             precision    recall  f1-score   support

          0       0.54      0.59      0.56       163
          1       0.50      0.46      0.48       149

avg / total       0.52      0.53      0.52       312



In [19]:
accuracy_score(yt, yp)

0.52564102564102566

In [20]:
print '--------- TRAIN -----------------'

--------- TRAIN -----------------


In [21]:
ytp = model.predict(X_train_word_average)
ytt = train_w2v_data['neuroticism_m']

In [22]:
print skmetrics.classification_report(y_true=ytt, y_pred=ytp)

             precision    recall  f1-score   support

          0       1.00      0.99      1.00       372
          1       0.99      1.00      1.00       355

avg / total       1.00      1.00      1.00       727



In [23]:
print accuracy_score(ytt, ytp)

0.997248968363
